In [1]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [2]:
adata4 = sc.read_h5ad('datasets/merfish/H20.30.001.STG.4000.expand.rep1.h5ad')
adata4.obs['coord'] = np.sqrt((adata4.obsm['spatial'][:,0]+3500) ** 2 + (adata4.obsm['spatial'][:,1]-2000) ** 2)
adata4.obs['niche'] = 'N6'
adata4.obs['niche'][adata4.obs['coord']> 3500] = 'N5'
adata4.obs['niche'][adata4.obs['coord']> 4700] = 'N4'
adata4.obs['niche'][adata4.obs['coord']> 5600] = 'N3'
adata4.obs['niche'][adata4.obs['coord']> 7000] = 'N2'
adata4.obs['niche'][adata4.obs['coord']> 7900] = 'N1'

adata5 = sc.read_h5ad('datasets/merfish/H20.30.001.STG.4000.expand.rep2.h5ad')
adata5.obs['coord'] = -((adata5.obsm['spatial'][:,0]+3500) + 0.4 * (adata5.obsm['spatial'][:,1]-2000)) +5000
adata5.obs['coord2'] = -((adata5.obsm['spatial'][:,0]+3500) + 0.3 * (adata5.obsm['spatial'][:,1]-2000)) +5000
adata5.obs['niche'] = 'N6'
adata5.obs['niche'][adata5.obs['coord']> 1800] = 'N5'
adata5.obs['niche'][adata5.obs['coord2']> 3000] = 'N4'
adata5.obs['niche'][adata5.obs['coord2']> 3400] = 'N3'
adata5.obs['niche'][adata5.obs['coord']> 5200] = 'N2'
adata5.obs['niche'][adata5.obs['coord']> 5600] = 'N1'

adata6 = sc.read_h5ad('datasets/merfish/H20.30.001.STG.4000.expand.rep3.h5ad')
adata6.obs['coord'] = adata6.obsm['spatial'][:,1]
adata6.obs['niche'] = 'N6'
adata6.obs['niche'][adata6.obs['coord']> 2500] = 'N5'
adata6.obs['niche'][adata6.obs['coord']> 4000] = 'N4'
adata6.obs['niche'][adata6.obs['coord']> 4400] = 'N3'

adata4.obs['batch'] = 'H20.30.001.STG.4000.rep1'
adata5.obs['batch'] = 'H20.30.001.STG.4000.rep2'
adata6.obs['batch'] = 'H20.30.001.STG.4000.rep3'

adata_stg = sc.concat([adata4,adata5,adata6])
adata_stg.obs_names_make_unique()
adata_stg

/tmp/ipykernel_2135087/481034249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata4.obs['niche'][adata4.obs['coord']> 3500] = 'N5'
/tmp/ipykernel_2135087/481034249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata4.obs['niche'][adata4.obs['coord']> 4700] = 'N4'
/tmp/ipykernel_2135087/481034249.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata4.obs['niche'][adata4.obs['coord']> 5600] = 'N3'
/tmp/ipykernel_2135087

AnnData object with n_obs × n_vars = 14924 × 4000
    obs: 'name', 'global.x', 'global.y', 'adjusted.x', 'adjusted.y', 'fov.x', 'fov.y', 'cluster_L1', 'cluster_L2', 'cluster_L3', 'coord', 'niche', 'batch'
    obsm: 'spatial'

In [4]:
from simvi.model import SimVI
SimVI.setup_anndata(adata_stg,batch_key='batch')
edge_index = SimVI.extract_edge_index(adata_stg,n_neighbors=10,batch_key='batch')

## likelihood

from pytorch_lightning.utilities.seed import seed_everything
for i in range(5):
    for reg_to_use in ['mi','mmd']:
        for reg_strength in [0,1,2,5,10,25]:
            if reg_to_use == 'mmd':
                reg_strength_ = reg_strength * 100
            else:
                reg_strength_ = reg_strength
            seed_everything(i)
            model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=reg_strength_,permutation_rate=0,reg_to_use=reg_to_use)
            train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True,device='cuda:0')
            adata_stg.obsm['simvi_z_'+reg_to_use+str(reg_strength_)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
            adata_stg.obsm['simvi_s_'+reg_to_use+str(reg_strength_)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

    for permutation_rate in [0,0.1,0.2,0.5]:
        seed_everything(i)
        model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=permutation_rate,reg_to_use='mi')
        train_loss, val_loss = model.train(edge_index,max_epochs=100,mae_epochs=25,batch_size=1000,use_gpu=True,device='cuda:0')
        adata_stg.obsm['simvi_z_p'+str(permutation_rate)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
        adata_stg.obsm['simvi_z_p'+str(permutation_rate)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
    
    for n_layers in [1,2]:
        seed_everything(i)
        model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=0,reg_to_use='mi',n_layers = n_layers)
        train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True,device='cuda:0')
        adata_stg.obsm['simvi_z'+'_nl_'+str(n_layers)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
        adata_stg.obsm['simvi_s'+'_nl_'+str(n_layers)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
        
    for dis_to_use in ['nb','zinb']:
        seed_everything(i)
        model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=0,reg_to_use='mi',dis_to_use = dis_to_use)
        train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True,device='cuda:0')
        adata_stg.obsm['simvi_z'+dis_to_use+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
        adata_stg.obsm['simvi_s'+dis_to_use+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
    
    for n_intrinsic in [10,20,40]:
        for n_spatial in [int(0.25 * n_intrinsic),int(0.5 * n_intrinsic),int(n_intrinsic)]:
            seed_everything(i)
            model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=0,reg_to_use='mi',n_intrinsic=n_intrinsic, n_spatial = n_spatial)
            train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True,device='cuda:0')
            adata_stg.obsm['simvi_z'+'_'+str(n_intrinsic)+str(n_spatial)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
            adata_stg.obsm['simvi_s'+'_'+str(n_intrinsic)+str(n_spatial)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
        
    for batch_size in [500,1000,2000]:
        seed_everything(i)
        model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=0,reg_to_use='mi')
        train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size= batch_size,use_gpu=True,device='cuda:0')
        adata_stg.obsm['simvi_z'+'_bs'+str(batch_size)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
        adata_stg.obsm['simvi_s'+'_bs'+str(batch_size)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
        
    for mae_epochs in [25,50,75]:
        seed_everything(i)
        model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=0.5,reg_to_use='mi')
        train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size= 1000,mae_epochs=mae_epochs,use_gpu=True,device='cuda:0')
        adata_stg.obsm['simvi_z'+'_mae'+str(mae_epochs)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
        adata_stg.obsm['simvi_s'+'_mae'+str(mae_epochs)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
    
    for kl_weight in [0.01,0.5,1]:
        for kl_gatweight in [0.01,0.5,1]:
            seed_everything(i)
            model = SimVI(adata_stg,kl_weight=kl_weight,kl_gatweight=kl_gatweight,lam_mi=5,permutation_rate=0,reg_to_use='mi')
            train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size= 1000,use_gpu=True,mae_epochs=0,device='cuda:0')
            adata_stg.obsm['simvi_z'+'_klw'+str(kl_weight)+str(kl_gatweight)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
            adata_stg.obsm['simvi_s'+'_klw'+str(kl_weight)+str(kl_gatweight)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

        
    for n_neighbors in [5,10,20,30]:
        SimVI.setup_anndata(adata_stg,batch_key='batch')
        edge_index = SimVI.extract_edge_index(adata_stg,n_neighbors=n_neighbors,batch_key='batch')
        seed_everything(i)
        model = SimVI(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=5,permutation_rate=0,reg_to_use='mi')
        train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True,device='cuda:0')
        adata_stg.obsm['simvi_z'+'_n'+str(n_neighbors)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
        adata_stg.obsm['simvi_s'+'_n'+str(n_neighbors)+'_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)        
adata_stg.write('stg_tuning_full_rsfixed.h5ad')

Global seed set to 0
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Global seed set to 0
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.19it/s, train_loss=732, val_loss=744.97833]
Global seed set to 0
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:30<00:00,  3.31it/s, train_loss=733, val_loss=745.0967]
Global seed set to 0
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:30<00:00,  3.33it/s, train_loss=733, val_loss=745.44495]
Global seed set to 0
Epoch 100/100: 100%|██████████████

Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.83it/s, train_loss=733, val_loss=744.45123]
Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.83it/s, train_loss=734, val_loss=745.18915]
Global seed set to 1
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.38it/s, train_loss=735, val_loss=747.6929]
Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:30<00:00,  3.31it/s, train_loss=726, val_loss=741.4373]
Global seed set to 2
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.34it/s, train_loss=735, val_loss=750.0531]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.39it/s, train_loss=734, val_loss=745.44904]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.85it/s, train_loss=733, val_loss=745.7234]
Global seed set to 4
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.87it/s, train_loss=733, val_loss=745.6449]
Global seed set to 4
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.90it/s, train_loss=734, val_loss=748.3452]
Global seed set to 4
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████